# Анализ данных с Pandas

In [ ]:
import pandas as pd
import numpy as np

## Структура

In [ ]:
q = pd.DataFrame(data={'a': ['abc', 'bcd', 'def'], 'b': [1,2,3], 'c': [100, 300, 200]}, index=[8,9,10])
q

In [ ]:
q.index

In [ ]:
q.columns

In [ ]:
q.a

In [ ]:
q['a']

In [ ]:
type(q.a)

In [ ]:
q[:2]

In [ ]:
q[ ['a', 'b'] ]

In [ ]:
q[['a', 'b']][:2]

## Чтение и просмотр

Данные соцопросов Census Income, с которыми будем работать:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/yakushinav/omo/main/data/adult_data.csv')

* __age__: continuous.
* __workclass__: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* __fnlwgt__: continuous.
* __education__: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* __education-num__: continuous.
* __marital-status__: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* __occupation__: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* __relationship__: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* __race__: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* __sex__: Female, Male.
* __capital-gain__: continuous.
* __capital-loss__: continuous.
* __hours-per-week__: continuous.
* __native-country__: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

`head()` показывает первые 5 строк датафрейма.

`tail()` - последние 5.

`head(N)` показывает первые N строк датафрейма.

`tail(N)` - последние N.

In [ ]:
df.head(3)

## Общее

Размер:

In [ ]:
df.shape

Столбцы:

In [ ]:
df.columns

Информация содержит данные о типе колонок, степени их заполненности (в нашем случае пропусков в данных нет), типе данных каждой из колонок, а также потреблении памяти:

In [ ]:
df.info()

Метод `describe` содержит основные статистики по числовым столбцам данных:
* количество
* среднее
* среднеквадратичное отклонение
* минимум, максимум
* медиану
* первый и третий квартили

In [ ]:
df.describe()

Если хочется посмотреть и на нечисловые признаки, то нужно указать их тип в параметре `include`. Всего есть совсем немного основных:
* `bool` - логический
* `object` - общий нечисловой, в большинстве случаев это строки
* `numeric` - числовой, включающий как числа с плавающей запятой, так и целые

In [ ]:
df.describe(include=['object', 'bool'])

Полезным методом является `value_counts()`, считающим распределение данных по категориям:

In [ ]:
df.education.value_counts()

Для отображения в долях существует флажок `normalize`:

In [ ]:
df.education.value_counts(normalize=True)

Можно строить простые диаграммы

In [ ]:
df.education.value_counts().plot.bar(); #(kind='bar');

## Индексация и фильтры

Весь датафрейм:

In [ ]:
print(df)

Только те, чей возраст меньше 25 лет, но больший 10:

In [ ]:
df[(df.age < 25) & (df.age > 10)]

Только те, кто получают больше 50 000$:

In [ ]:
df[df.salary == '>50K']

In [ ]:
df[df.salary == '<=50K'].sex.value_counts(normalize=False)

In [ ]:
df[df.salary == '>50K'].sex.value_counts(normalize=True)

## Сортировка

In [ ]:
df.sort_values(by=['education'])

Отсортируем сначала по полу, внутри пола - по возрасту, но в обратном порядке

In [ ]:
df.sort_values(by=['sex', 'age'], ascending=[True, False])

## Применение функций

Применить функцию к каждому столбцу или каждой строке можно с помощью `apply`:

In [ ]:
t = pd.DataFrame(data = {'a': [1,2,3], 'b': [4,5,6]})
t

In [ ]:
t['c'] = t.apply(lambda row: row['a'] + row['b'], axis=1) # построчно

In [ ]:
t

Применение функции к каждой __ячейке__ реализовывается с помощью `map`:

In [ ]:
renaming = {'>50K': 1, '<=50K': 0}
df.salary = df.salary.map(renaming)

In [ ]:
df.salary.value_counts()

In [ ]:
renaming = {1:'> 50K', 0: '<= 50K'}
df.salary = df.salary.map(renaming)

__apply__ к нескольким столбцам сразу называется __applymap__:

In [ ]:
df['sex'] = df['sex'].apply(lambda x: x.lower())
df['relationship'] = df['relationship'].apply(lambda x: x.lower())

In [ ]:
# same as above:
cols = ['sex', 'relationship']
df[cols] = df[cols].applymap(lambda x: x.lower())

У строковых столбцов есть хитрое поле __str__, которое возвращает наружу как будто строку (синтаксис тот же), но применяет к каждому элементу столбца.

При больших объемах данных это будет быстрее, потому что `.str` под капотом векторизует операции со строками. Для дат можно использовать `.dt`.

In [ ]:
df.sex.apply(lambda x: x.upper());

In [ ]:
# same as above
df.sex.str.upper();

## Дубликаты

In [ ]:
q = pd.DataFrame(data={'a': [1,2,3,1,1], 'b': [1,2,3,2,1], 'c':[10,20,30,40,40]})
q

In [ ]:
q.duplicated(['a'])

In [ ]:
q.duplicated(['a', 'b']) # - уже оценивает кортежи значений из нескольких колонок

In [ ]:
q.drop_duplicates(subset=['a'])

In [ ]:
q.drop_duplicates(subset=['a', 'b'])

## Группировка

In [ ]:
df.groupby(['salary']);

__Важно__: groupby-объекты - не копии частей датафрейма!

In [ ]:
df.groupby(['salary']).describe(include=['object', 'bool'])

In [ ]:
df.groupby('salary')

In [ ]:
df.sex.value_counts()

In [ ]:
df.groupby('salary').sex.value_counts().plot.bar();

## Таблицы сопряженности и melt

### Таблицы сопряженности

In [ ]:
pd.crosstab(df.salary, df.sex).plot(kind='bar');

In [ ]:
pd.crosstab(df.salary, df.sex, normalize=True)

In [ ]:
pd.crosstab(df.salary, df['marital-status'], normalize=True).plot.bar();

### Melt - превращение столбцов в строки

In [ ]:
t = pd.DataFrame(data={'name': ['PC', 'PlayStation4'],
                   '1W-2014': [100, 300],
                   '2W-2014': [150, 350]})
t

In [ ]:
t2 = pd.melt(t, id_vars=['name'], value_name='cost', var_name='timestamp')
t2

## Изменение таблиц

In [ ]:
df.head(2)

### Добавление нового столбца

In [ ]:
df['working_days'] = df['hours-per-week'] / 8.0

In [ ]:
df.head(2)

In [ ]:
df['constant'] = 1

In [ ]:
df = df.assign(anotherConstant = lambda dataframe: dataframe.constant*2)

In [ ]:
df.head(2)

### Удаление строк и столбцов

In [ ]:
df.drop(['constant', 'anotherConstant'], axis=1, inplace=True)

__NB__: У большинства операторов pandas есть флажок `inplace`. Если он выставлен в `False` (по умолчанию именно так), то изменения, например, как `drop` в примере выше, не будут изменять таблицу, а вернут измененную копию.

In [ ]:
df.drop([1,2]).head(3) # => удаляет строки

### loc, iloc

In [ ]:
df.loc[0, 'workclass'] # получение конкретной ячейки по индексу и названию столбца

In [ ]:
df.iloc[0, 1] # получение конкретной ячейки по номеру строки и номеру столбца, вне зависимости от индексов

__Важно__: можно и получать строки по маскам:

In [ ]:
df.loc[df.age % 5 == 0, 'workclass'].head()

__Важно__: Основываясь на этом, можно присваивать другим значениям:

In [ ]:
df.loc[df.age % 2 == 0, 'workclass'] = 'ЧЕТНЫЙ'

In [ ]:
df.loc[:5, 'education'] = 'NO EDUCATION'

In [ ]:
df.head(7)

### Переименование столбцов

In [ ]:
df.columns = [c.upper() for c in df.columns]

In [ ]:
df.head(1)

In [ ]:
df.columns = [c.lower() for c in df.columns]

In [ ]:
df = df.rename(columns={'marital-status': 'marital_status'})
                        # словарь старое-новое значение

## Работа с пропущенными (NaN) значениями

In [ ]:
test = pd.DataFrame(data={'name': ['John', 'Bill', 'Jennifer', 'Max'],
                          'salary': [90, np.nan, 95, 130],
                          'age': [27, 30, np.nan, np.nan],
                          'status': [np.nan, 'married', np.nan, 'married']
                         })
test = test[['name', 'age', 'salary', 'status']]
test

### Отображение

In [ ]:
test[test['age'].isnull()]

In [ ]:
test[test['age'].notnull()]

In [ ]:
test[test.isnull().any(axis=1)] # !

In [ ]:
test[~test.isnull().any(axis=1)]

### Заполнение

Прежде всего, можно заполнить некоторым константным значением:

In [ ]:
dashtest = test.fillna('-')
dashtest

Важно заметить, что теперь слобцы возраста и зарплаты более не числовые:

In [ ]:
print(test.dtypes, '\n\n', dashtest.dtypes)

In [ ]:
test.fillna(method='ffill')

In [ ]:
test.fillna(method='bfill')

In [ ]:
test.fillna(method='ffill').fillna(method='bfill')

In [ ]:
test['salary'] = test[['salary']].fillna(test.salary.median())
test

In [ ]:
test['age'] = test[['salary']].fillna(25)
test['status'] = test[['status']].fillna(test.status.mode()[0])
test

## Краткий конспект

- df.shape - размер
- df.head(N), df.tail(N), - просмотр
- df[массив имен колонок] - часть (slice) датафрейма
- df['имя колонки'] - одна колонка (можно и через точку: df.имя_колонки)
- df.unique, df.nunique - просмотр уникальных значений и их количества
- df.value_counts - сколько разных значений в колонке
- df.имя колонки.apply - применяет функцию к значениям в столбце
- df.fillna - заполняет NaN
- df.dropna - удаляет строки/столбцы с NaN
- df.duplicated - показывает строки, являющиеся дупликатами других
- df.drop_duplicates - удаляет строки-дупликаты
- df[df.isnull().any(axis=1)] - вывести строки, содержащие NaN
- df[df.имя колонки.isnull().any(axis=1)] - вывести только те строки, где в указанной колонке есть пропуски
- df.groupby - группировка по некоторому признаку

## Задание

**1.** Загрузите csv-файл `https://raw.githubusercontent.com/yakushinav/omo/main/data/fishing.csv` в переменную fishing_data. Описание данных см. [здесь](https://github.com/yakushinav/journ/blob/main/fishing.md).

In [1]:
import pandas as pd
fishing_data = pd.read_csv("https://raw.githubusercontent.com/yakushinav/omo/main/data/fishing.csv")

**2.** Выведите описательные статистики для всех числовых (integer, float) переменных в датафрейме.

In [2]:
print(fishing_data.describe())

        Unnamed: 0        price        catch       pbeach        ppier  \
count  1182.000000  1182.000000  1182.000000  1182.000000  1182.000000   
mean    591.500000    52.081975     0.389368   103.422005   103.422005   
std     341.358316    53.829970     0.560596   103.641042   103.641042   
min       1.000000     1.290000     0.000200     1.290000     1.290000   
25%     296.250000    15.870000     0.036100    26.656500    26.656500   
50%     591.500000    37.896000     0.164300    74.628000    74.628000   
75%     886.750000    67.513000     0.533300   144.144000   144.144000   
max    1182.000000   666.110000     2.310100   843.186000   843.186000   

             pboat     pcharter       cbeach        cpier        cboat  \
count  1182.000000  1182.000000  1182.000000  1182.000000  1182.000000   
mean     55.256570    84.379244     0.241011     0.162224     0.171215   
std      62.713444    63.544650     0.190752     0.160390     0.209789   
min       2.290000    27.290000     0

**3.** Выберите из таблицы только строки, которые соответствуют респондентам, которые выбрали рыбалку на берегу (`beach`).

In [3]:
beach_fishing = fishing_data[fishing_data['mode'] == 'beach']
print(beach_fishing)

      Unnamed: 0   mode    price   catch   pbeach    ppier    pboat  pcharter  \
6              7  beach   51.934  0.0678   51.934   51.934  191.930   220.930   
14            15  beach   74.514  0.2537   74.514   74.514   74.514    93.014   
16            17  beach   48.114  0.1049   48.114   48.114   33.534    57.534   
25            26  beach   67.784  0.5333   67.784   67.784   17.862    52.862   
33            34  beach    9.522  0.0678    9.522    9.522   78.292   107.292   
...          ...    ...      ...     ...      ...      ...      ...       ...   
1167        1168  beach   23.688  0.5333   23.688   23.688  139.496   174.496   
1173        1174  beach   62.634  0.0678   62.634   62.634  123.552   152.552   
1176        1177  beach  121.836  0.5333  121.836  121.836  203.346   238.346   
1178        1179  beach  235.436  0.5333  235.436  235.436  392.946   427.946   
1180        1181  beach   36.636  0.5333   36.636   36.636   61.146    96.146   

      cbeach   cpier   cboa

**4.** Выберите из таблицы строки, которые соответствуют респондентам, которые предпочитают рыбачить на лодке (`boat`) с доходом (`income`) ниже 3500. Сохраните результат в переменную `boat_data`.

In [4]:
boat_data = fishing_data[(fishing_data['mode'] == 'boat') & (fishing_data['income'] < 3500)]
print(boat_data)

      Unnamed: 0  mode    price   catch   pbeach    ppier    pboat  pcharter  \
9             10  boat   28.314  0.0233   28.314   28.314   28.314    46.814   
22            23  boat   15.134  0.1665  100.674  100.674   15.134    50.134   
32            33  boat    7.722  0.0023   33.462   33.462    7.722    31.722   
37            38  boat   80.608  0.0531   25.648   25.648   80.608   105.608   
40            41  boat   46.332  0.1643   10.296   10.296   46.332    75.332   
...          ...   ...      ...     ...      ...      ...      ...       ...   
1121        1122  boat   48.906  0.0732  204.204  204.204   48.906    78.406   
1129        1130  boat    3.290  0.0531   37.506   37.506    3.290    28.290   
1135        1136  boat   37.506  0.0102  101.990  101.990   37.506    72.506   
1141        1142  boat  132.990  0.0156    4.290    4.290  132.990   167.990   
1154        1155  boat   39.990  0.1574   30.186   30.186   39.990    63.990   

      cbeach   cpier   cboat  ccharter 

**5.** Переименуйте столбец `catch` в `c_rate`.

In [5]:
fishing_data = fishing_data.rename(columns={'catch': 'c_rate'})

**6.** Добавьте в таблицу столбец `log_income`, содержащий натуральный логарифм доходов респондентов.

In [6]:
import numpy as np
fishing_data['log_income'] = np.log(fishing_data['income'])

**7.** Сгруппируйте наблюдения в таблице по признаку тип рыбалки (`mode`) и выведите для каждого типа среднюю цену (`price`), которую респонденты готовы заплатить за рыбалку.

In [7]:
mean_price_by_mode = fishing_data.groupby('mode')['price'].mean()
print(mean_price_by_mode)

mode
beach      35.699493
boat       41.606813
charter    75.096942
pier       30.571326
Name: price, dtype: float64


**8.** Выведите на экран первые 8 строк таблицы. А теперь последние 8.

In [8]:
print("Первые 8 строк:\n", fishing_data.head(8))
print("\nПоследние 8 строк:\n", fishing_data.tail(8))

Первые 8 строк:
    Unnamed: 0     mode    price  c_rate   pbeach    ppier    pboat  pcharter  \
0           1  charter  182.930  0.5391  157.930  157.930  157.930   182.930   
1           2  charter   34.534  0.4671   15.114   15.114   10.534    34.534   
2           3     boat   24.334  0.2413  161.874  161.874   24.334    59.334   
3           4     pier   15.134  0.0789   15.134   15.134   55.930    84.930   
4           5     boat   41.514  0.1082  106.930  106.930   41.514    71.014   
5           6  charter   63.934  0.3975  192.474  192.474   28.934    63.934   
6           7    beach   51.934  0.0678   51.934   51.934  191.930   220.930   
7           8  charter   56.714  0.0209   15.134   15.134   21.714    56.714   

   cbeach   cpier   cboat  ccharter     income  log_income  
0  0.0678  0.0503  0.2601    0.5391  7083.3317    8.865500  
1  0.1049  0.0451  0.1574    0.4671  1249.9998    7.130899  
2  0.5333  0.4522  0.2413    1.0266  3749.9999    8.229511  
3  0.0678  0.0789 

**9.** Сохраните изменённый датафрейм в csv-файл.

In [10]:
fishing_data.to_csv('fishing_data_modified.csv', index=False)